In [28]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import pathlib
import subprocess
import numpy as np
from multiprocessing import Pool


In [58]:
class HBRad(object):
    def __init__(self, source_directory, n_processes=1):
        self.source_directory = pathlib.Path(source_directory)
        self.source_pts = self.find_pts()
        self.n_processes = n_processes
        self.sub_pts = self.split_pts_file()
        self.bat_file = self.source_directory / "commands.bat"
        self.bat_init = None
        self.sub_bat_files = self.split_bat_file()
    
    def find_pts(self):
        return list(pathlib.Path(self.source_directory).glob('**/*{}'.format(".pts")))[0]
    
    def split_bat_file(self):
        # Get header and find line where calculation starts
        with open(self.bat_file, "r") as f:
            bat_commands = [i.strip() for i in f.readlines()]
        for n, i in enumerate(bat_commands):
            if i == "": 
                header_idx = n
            if "start of the calculation for scene" in i:
                split_idx = n
            if " > analemma.oct" in i:
                analemma_idx = n
        
        # Put the initial smx commands into their own bat_init file
        self.bat_init = self.source_directory / "{0:}_init.bat".format(self.bat_file.stem)
        header = bat_commands[:split_idx] + [bat_commands[analemma_idx]]
        with open(self.bat_init, "w") as f:
            for i in header:
                if "echo" in i:
                    pass
                else:
                    f.write("{0:}\n".format(i))
        
        # Put the remainder of the bat commands into pts-set respective files
        sub_bat_files = []
        for i in range(self.n_processes):
            base_outstr = bat_commands[:header_idx] + bat_commands[split_idx:analemma_idx] + bat_commands[analemma_idx+1:]
            outstr = []
            for j in base_outstr:
                if "echo" in j:
                    pass
                else:
                    outstr.append(j.replace(self.source_pts.name, "{0:}_{1:03d}.pts".format(self.source_pts.stem, i)).replace(".dc", "_{0:03d}.dc".format(i)).replace(".rgb", "_{0:03d}.rgb".format(i)).replace(".ill", "_{0:03d}.ill".format(i)))
            sub_bat_file = self.source_directory / "{0:}_{1:03d}.bat".format(self.bat_file.stem, i)
            with open(sub_bat_file, "w") as f:
                f.write("\n".join(outstr))
            sub_bat_files.append(sub_bat_file)
        return sub_bat_files
        
    
    def split_pts_file(self):
        sub_pts = []
        with open(self.source_pts, "r") as f:
            src_pts = [i.strip() for i in f.readlines()]
        src_pts_split = chunks(src_pts, self.n_processes)
        
        for i in range(self.n_processes):
            sub_pts_file = self.source_directory / "{0:}_{1:03d}.pts".format(self.source_pts.stem, i)
            with open(sub_pts_file, "w") as f:
                f.write("\n".join(src_pts_split[i]))
            sub_pts.append(sub_pts_file)
        return sub_pts
    
    def __repr__(self):
        return "{klass}\n{attrs}".format(
            klass=self.__class__.__name__,
            attrs="\n".join("- {}: {!r}".format(k, v) for k, v in self.__dict__.items()),
        )

def chunks(enumerable, n=None):
    enumerable = np.array(enumerable)
    return [i for i in np.array_split(enumerable, n)]

HBRad(
    source_directory=r"C:\Users\tgerrish\Desktop\_0Ghadan21\EXAMPLE\gridbased_annual", 
    n_processes=2
)

HBRad
- source_directory: WindowsPath('C:/Users/tgerrish/Desktop/_0Ghadan21/EXAMPLE/gridbased_annual')
- source_pts: WindowsPath('C:/Users/tgerrish/Desktop/_0Ghadan21/EXAMPLE/gridbased_annual/EXAMPLE.pts')
- n_processes: 2
- sub_pts: [WindowsPath('C:/Users/tgerrish/Desktop/_0Ghadan21/EXAMPLE/gridbased_annual/EXAMPLE_000.pts'), WindowsPath('C:/Users/tgerrish/Desktop/_0Ghadan21/EXAMPLE/gridbased_annual/EXAMPLE_001.pts')]
- bat_file: WindowsPath('C:/Users/tgerrish/Desktop/_0Ghadan21/EXAMPLE/gridbased_annual/commands.bat')
- bat_init: WindowsPath('C:/Users/tgerrish/Desktop/_0Ghadan21/EXAMPLE/gridbased_annual/commands_init.bat')
- sub_bat_files: [WindowsPath('C:/Users/tgerrish/Desktop/_0Ghadan21/EXAMPLE/gridbased_annual/commands_000.bat'), WindowsPath('C:/Users/tgerrish/Desktop/_0Ghadan21/EXAMPLE/gridbased_annual/commands_001.bat')]